### *Импорты* ###

In [1]:
import numpy as np
import pandas as pd
import joblib

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import load_model
from datetime import datetime, timedelta

### *Загрузка файла и стандартизаторов данных* ###

In [2]:
data = pd.read_csv('../ann/../../db/basel(1940-2023).csv')

scaler_temperature = joblib.load('scalers/scaler_temperature.joblib')
scaler_relative_humidity = joblib.load('scalers/scaler_relative_humidity.joblib')
scaler_wind_speed = joblib.load('scalers/scaler_wind_speed.joblib')

### *Загрузка моделей* ###

In [3]:
model_temperature = load_model('best_models/best_model_temperature.keras')
model_relative_humidity = load_model('best_models/best_model_relative_humidity.keras')
model_wind_speed = load_model('best_models/best_model_wind_speed.keras')

### *Классы, хранящие данные прогноза погоды* ###

In [4]:
class HourlyForecast:
    def __init__(self, year, month, day, hour, temperature, relative_humidity, wind_speed):
        self.year = year
        self.month = month
        self.day = day
        self.hour = hour
        self.temperature = temperature[0][0]
        self.relative_humidity = relative_humidity[0][0]
        self.wind_speed = wind_speed[0][0]

In [5]:
class DailyForecast:
    def __init__(self, year, month, day, temperature):
        self.year = year
        self.month = month
        self.day = day
        self.temperature = temperature[0][0]

### *Функции, выполняющие рассчеты* ###

In [6]:
# Функция, предсказывающая погоду на ближайщие от текущей даты 5 часов и 5 дней
def predict_weather(date):
    year, month, day, hour = map(int, date.split('.'))
    current_date = datetime(year, month, day, hour)
    current_hour_data = get_current_hour_data(year, month, day, hour)

    hourly_forecast = []
    daily_forecast = []
    for i in range(5):
        target_hour = current_date + timedelta(hours=i)
        target_day = current_date + timedelta(days=i)

        hourly_weather_variations = get_last_5_hours_data(target_hour.year, target_hour.month, target_hour.day, target_hour.hour)
        daily_weather_variations = get_last_5_days_data(target_day.year, target_day.month, target_day.day, target_day.hour)

        hourly_temperature_prediction = predict_temperature_next_hour(current_hour_data, hourly_weather_variations)
        
        hourly_relative_humidity_prediction = predict_relative_humidity_next_hour(current_hour_data, 
                                                                                  hourly_weather_variations, 
                                                                                  daily_weather_variations)
        
        hourly_wind_speed_prediction = predict_wind_speed_next_hour(current_hour_data, 
                                                                    hourly_weather_variations, 
                                                                    daily_weather_variations)
        
        hourly_forecast.append(HourlyForecast(target_hour.year, 
                                              target_hour.month, 
                                              target_hour.day, 
                                              target_hour.hour, 
                                              hourly_temperature_prediction,
                                              hourly_relative_humidity_prediction,
                                              hourly_wind_speed_prediction))
        
        daily_temperature_prediction = predict_temperature_next_day(current_hour_data, 
                                                                    hourly_weather_variations, 
                                                                    daily_weather_variations)
        
        daily_forecast.append(DailyForecast(target_hour.year, 
                                            target_hour.month, 
                                            target_hour.day, 
                                            daily_temperature_prediction))

    return {'hourly': hourly_forecast, 'daily': daily_forecast}

In [7]:
def get_current_hour_data(year, month, day, hour):
    current_hour_data = data[(data['Year'] == year) &
                             (data['Month'] == month) &
                             (data['Day'] == day) &
                             (data['Hour'] == hour)]
    
    # Если данные о текущем часе неизвестны, будут взяты данные прошлого года
    while current_hour_data.empty:
        year -= 1
        
        current_hour_data = data[(data['Year'] == year) &
                         (data['Month'] == month) &
                         (data['Day'] == day) &
                         (data['Hour'] == hour)]
        
    return current_hour_data

In [8]:
def get_last_5_hours_data(year, month, day, hour):
    target_date = datetime(year, month, day, hour)
    target_dates = [target_date - timedelta(hours=i) for i in range(1, 6)]

    last_5_hours_data = pd.concat([data[(data['Year'] == date.year) &
                                       (data['Month'] == date.month) &
                                       (data['Day'] == date.day) &
                                       (data['Hour'] == date.hour)] for date in target_dates], ignore_index=True)
    
    # Если данные о последних 5 часах перед предсказываемой датой неизвестны, будут взяты данные прошлого года
    while last_5_hours_data.empty:
        target_date -= timedelta(days=365)
        target_dates = [target_date - timedelta(hours=i) for i in range(1, 6)]

        last_5_hours_data = pd.concat([data[(data['Year'] == date.year) &
                                           (data['Month'] == date.month) &
                                           (data['Day'] == date.day) &
                                           (data['Hour'] == date.hour)] for date in target_dates], ignore_index=True)

    return last_5_hours_data

In [9]:
def get_last_5_days_data(year, month, day, hour):
    target_date = datetime(year, month, day, hour)
    target_dates = [target_date - timedelta(days=i) for i in range(1, 6)]
    
    last_5_days_data = pd.concat([data[(data['Year'] == date.year) &
                                       (data['Month'] == date.month) &
                                       (data['Day'] == date.day)] for date in target_dates], ignore_index=True)
    
    # Если данные о последних 5 днях перед предсказываемой датой неизвестны, будут взяты данные прошлого года
    while last_5_days_data.empty:
        target_date -= timedelta(days=365)
        target_dates = [target_date - timedelta(days=i) for i in range(1, 6)]

        last_5_days_data = pd.concat([data[(data['Year'] == date.year) &
                                           (data['Month'] == date.month) &
                                           (data['Day'] == date.day)] for date in target_dates], ignore_index=True)

    return last_5_days_data

In [10]:
def predict_temperature_next_hour(current_hour_data, last_5_hours_data):
    current_hour_data = current_hour_data.drop(['Temperature [2 m elevation corrected]'], axis=1)
    last_5_hours_data = last_5_hours_data.drop(['Temperature [2 m elevation corrected]'], axis=1)
    
    middle_point = (current_hour_data + last_5_hours_data.mean()) / 2
    middle_point_scaled = scaler_temperature.transform(middle_point)
    
    prediction = model_temperature.predict(middle_point_scaled)
    return prediction

In [11]:
def predict_temperature_next_day(current_hour_data, last_5_hours_data, last_5_days_data):
    current_hour_data = current_hour_data.drop(['Temperature [2 m elevation corrected]'], axis=1)
    last_5_hours_data = last_5_hours_data.drop(['Temperature [2 m elevation corrected]'], axis=1)
    last_5_days_data = last_5_days_data.drop(['Temperature [2 m elevation corrected]'], axis=1)
    
    middle_point = (current_hour_data + last_5_hours_data.mean() + last_5_days_data.mean()) / 3
    middle_point_scaled = scaler_temperature.transform(middle_point)
    
    prediction = model_temperature.predict(middle_point_scaled)
    return prediction

In [12]:
def predict_relative_humidity_next_hour(current_hour_data, last_5_hours_data, last_5_days_data):
    current_hour_data = current_hour_data.drop(['Relative Humidity [2 m]'], axis=1)
    last_5_hours_data = last_5_hours_data.drop(['Relative Humidity [2 m]'], axis=1)
    last_5_days_data = last_5_days_data.drop(['Relative Humidity [2 m]'], axis=1)

    middle_point = (current_hour_data + last_5_hours_data.mean() + last_5_days_data.mean()) / 3
    middle_point_scaled = scaler_relative_humidity.transform(middle_point)
    
    prediction = model_relative_humidity.predict(middle_point_scaled)
    return prediction

In [13]:
def predict_wind_speed_next_hour(current_hour_data, last_5_hours_data, last_5_days_data):
    current_hour_data = current_hour_data.drop(['Wind Speed [10 m]'], axis=1)
    last_5_hours_data = last_5_hours_data.drop(['Wind Speed [10 m]'], axis=1)
    last_5_days_data = last_5_days_data.drop(['Wind Speed [10 m]'], axis=1)

    middle_point = (current_hour_data + last_5_hours_data.mean() + last_5_days_data.mean()) / 3
    middle_point_scaled = scaler_wind_speed.transform(middle_point)
    
    prediction = model_wind_speed.predict(middle_point_scaled)
    return prediction

In [14]:
def print_weather_report(weather_report):
    for hwr in weather_report['hourly']:
        print('Date: {}.{}.{} {}:00 \n Temperature: {} °C \n Relative humidity: {}% \n Wind speed: {} m/s \n'.format(
            hwr.year, hwr.month, hwr.day, hwr.hour, hwr.temperature, hwr.relative_humidity, hwr.wind_speed))
        
    for dwr in weather_report['daily']:
        print('Date: {}.{}.{} \n Temperature: {} °C \n'.format(dwr.year, dwr.month, dwr.day, dwr.temperature))

In [15]:
weather_report = predict_weather('2023.12.21.14')

1/1 [==============================] - 0s 56ms/step


In [16]:
print_weather_report(weather_report)

Date: 2023.12.21 14:00 
 Temperature: 9.493486404418945 °C 
 Relative humidity: 92.62919616699219% 
 Wind speed: 2.1957242488861084 m/s 

Date: 2023.12.21 15:00 
 Temperature: 9.455698013305664 °C 
 Relative humidity: 92.37496185302734% 
 Wind speed: 2.210142135620117 m/s 

Date: 2023.12.21 16:00 
 Temperature: 9.46554946899414 °C 
 Relative humidity: 91.54730987548828% 
 Wind speed: 2.440941095352173 m/s 

Date: 2023.12.21 17:00 
 Temperature: 9.530021667480469 °C 
 Relative humidity: 90.7882308959961% 
 Wind speed: 2.6804609298706055 m/s 

Date: 2023.12.21 18:00 
 Temperature: 9.621610641479492 °C 
 Relative humidity: 90.14554595947266% 
 Wind speed: 2.782921314239502 m/s 

Date: 2023.12.21 
 Temperature: 6.565884113311768 °C 

Date: 2023.12.21 
 Temperature: 7.288824081420898 °C 

Date: 2023.12.21 
 Temperature: 8.319143295288086 °C 

Date: 2023.12.21 
 Temperature: 9.397512435913086 °C 

Date: 2023.12.21 
 Temperature: 9.893287658691406 °C 

